# Multi-Task Gaussian Process Tutorial

This notebook demonstrates how to use **MultitaskGP** from OpenAD-lib for multi-output prediction with uncertainty quantification.

## Overview

Multi-Task GPs are excellent for:
- Predicting multiple correlated outputs simultaneously
- Quantifying prediction uncertainty (confidence intervals)
- Learning correlations between output tasks
- Working with limited training data

## 1. Setup and Imports

In [ ]:
import sys
import os

# Add library to path if not installed
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'src'))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import MultitaskGP Model
from openad_lib.models.ml import MultitaskGP

print("Imports successful!")

## 2. Load and Explore Data

In [ ]:
# Load sample data
DATA_DIR = os.path.join(os.path.dirname(os.getcwd()), 'src', 'openad_lib', 'data')
data_path = os.path.join(DATA_DIR, 'sample_ad_process_data.csv')

data = pd.read_csv(data_path)

print(f"Dataset shape: {data.shape}")
print(f"\nColumns:")
print(data.columns.tolist())

data.head()

In [ ]:
# Explore the data
print("=== Data Statistics ===")
data.describe()

In [ ]:
# Visualize outputs
output_cols = ['SCODout', 'VFAout', 'Biogas']

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for i, col in enumerate(output_cols):
    if col in data.columns:
        axes[i].plot(data['time'], data[col], 'b-', linewidth=1)
        axes[i].set_xlabel('Time')
        axes[i].set_ylabel(col)
        axes[i].set_title(f'{col} Over Time')
        axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Correlation between outputs
output_data = data[output_cols]
corr = output_data.corr()

print("=== Output Correlations ===")
print(corr)

fig, ax = plt.subplots(figsize=(6, 5))
im = ax.imshow(corr, cmap='coolwarm', vmin=-1, vmax=1)
ax.set_xticks(range(len(output_cols)))
ax.set_yticks(range(len(output_cols)))
ax.set_xticklabels(output_cols)
ax.set_yticklabels(output_cols)
plt.colorbar(im)
plt.title('Output Correlations')
plt.show()

## 3. Prepare Multi-Output Data

In [ ]:
# Define inputs and outputs
input_cols = ['time', 'D', 'SCODin', 'OLR']
output_cols = ['SCODout', 'VFAout', 'Biogas']

X = data[input_cols].values
Y = data[output_cols].values

print(f"Inputs: {input_cols}")
print(f"Outputs: {output_cols}")
print(f"\nX shape: {X.shape}")
print(f"Y shape: {Y.shape}")

In [ ]:
# Split data (temporal order)
train_size = int(len(X) * 0.8)

X_train = X[:train_size]
X_test = X[train_size:]
Y_train = Y[:train_size]
Y_test = Y[train_size:]

print(f"Training samples: {len(X_train)}")
print(f"Testing samples: {len(X_test)}")

## 4. Create and Train MTGP Model

In [ ]:
# Create MTGP model
mtgp = MultitaskGP(
    num_tasks=3,          # 3 outputs (SCOD, VFA, Biogas)
    num_latents=3,        # 3 latent functions (LMC)
    n_inducing=50,        # Inducing points
    learning_rate=0.1,
    log_transform=True    # Log-transform outputs
)

print(f"MTGP Model Configuration:")
print(f"  Number of tasks: {mtgp.num_tasks}")
print(f"  Number of latents: {mtgp.num_latents}")
print(f"  Inducing points: {mtgp.n_inducing}")
print(f"  Device: {mtgp.device}")

In [ ]:
# Train the model
print("Training MTGP model...\n")

mtgp.fit(X_train, Y_train, epochs=150, verbose=True)

print("\nTraining complete!")

In [ ]:
# Plot training loss
plt.figure(figsize=(10, 5))
plt.plot(mtgp.training_losses, 'b-', linewidth=1)
plt.xlabel('Epoch')
plt.ylabel('Negative ELBO')
plt.title('Training Loss Over Epochs')
plt.grid(True, alpha=0.3)
plt.show()

## 5. Predictions with Uncertainty

In [ ]:
# Get predictions with confidence intervals
mean, lower, upper = mtgp.predict(X_test, return_std=True)

print(f"Predictions shape: {mean.shape}")
print(f"Lower bound shape: {lower.shape}")
print(f"Upper bound shape: {upper.shape}")
print("\n95% confidence intervals available for all outputs!")

In [ ]:
# Evaluate performance
metrics = mtgp.evaluate(X_test, Y_test, task_names=output_cols)

print("=== Test Metrics ===")
for task, m in metrics.items():
    print(f"\n{task}:")
    print(f"  RMSE: {m['rmse']:.2f}")
    print(f"  MAE:  {m['mae']:.2f}")
    print(f"  R²:   {m['r2']:.3f}")

## 6. Visualize Predictions with Uncertainty

In [ ]:
# Plot predictions with confidence intervals
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for i, (name, ax) in enumerate(zip(output_cols, axes)):
    x_axis = range(len(Y_test))
    
    # Actual values
    ax.plot(x_axis, Y_test[:, i], 'b.', label='Actual', markersize=8)
    
    # Predicted mean
    ax.plot(x_axis, mean[:, i], 'r-', label='Predicted', linewidth=2)
    
    # 95% confidence interval
    ax.fill_between(x_axis, lower[:, i], upper[:, i], 
                    alpha=0.3, color='red', label='95% CI')
    
    ax.set_xlabel('Sample Index')
    ax.set_ylabel(name)
    ax.set_title(f'{name} (R² = {metrics[name]["r2"]:.3f})')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Training predictions
train_mean, train_lower, train_upper = mtgp.predict(X_train, return_std=True)

fig, axes = plt.subplots(2, 3, figsize=(15, 8))

for i, name in enumerate(output_cols):
    # Training
    axes[0, i].plot(Y_train[:, i], train_mean[:, i], 'b.', alpha=0.5)
    axes[0, i].plot([Y_train[:, i].min(), Y_train[:, i].max()], 
                    [Y_train[:, i].min(), Y_train[:, i].max()], 'r--')
    axes[0, i].set_xlabel(f'Actual {name}')
    axes[0, i].set_ylabel(f'Predicted {name}')
    axes[0, i].set_title(f'Training: {name}')
    
    # Testing
    axes[1, i].plot(Y_test[:, i], mean[:, i], 'g.', alpha=0.5)
    axes[1, i].plot([Y_test[:, i].min(), Y_test[:, i].max()], 
                    [Y_test[:, i].min(), Y_test[:, i].max()], 'r--')
    axes[1, i].set_xlabel(f'Actual {name}')
    axes[1, i].set_ylabel(f'Predicted {name}')
    axes[1, i].set_title(f'Testing: {name}')

plt.tight_layout()
plt.show()

## 7. Uncertainty Analysis

In [ ]:
# Calculate uncertainty width
uncertainty = upper - lower

print("=== Uncertainty Analysis ===")
for i, name in enumerate(output_cols):
    # Check if actual values fall within CI
    within_ci = (Y_test[:, i] >= lower[:, i]) & (Y_test[:, i] <= upper[:, i])
    coverage = np.mean(within_ci) * 100
    
    print(f"\n{name}:")
    print(f"  Mean uncertainty width: {np.mean(uncertainty[:, i]):.2f}")
    print(f"  95% CI coverage: {coverage:.1f}%")

In [ ]:
# Plot uncertainty distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for i, name in enumerate(output_cols):
    axes[i].hist(uncertainty[:, i], bins=20, edgecolor='black', alpha=0.7)
    axes[i].axvline(np.mean(uncertainty[:, i]), color='red', linestyle='--', 
                    label=f'Mean: {np.mean(uncertainty[:, i]):.2f}')
    axes[i].set_xlabel('Uncertainty Width')
    axes[i].set_ylabel('Frequency')
    axes[i].set_title(f'{name} Uncertainty Distribution')
    axes[i].legend()

plt.tight_layout()
plt.show()

## 8. Save and Load Model

In [ ]:
# Save the model
model_path = 'mtgp_ad_model.pt'
mtgp.save(model_path)
print(f"Model saved to {model_path}")

In [ ]:
# Load the model (requires sample data)
loaded_mtgp = MultitaskGP.load(model_path, X_train[:10])
print("Model loaded successfully!")

# Verify predictions
loaded_mean, _, _ = loaded_mtgp.predict(X_test[:5], return_std=True)
original_mean, _, _ = mtgp.predict(X_test[:5], return_std=True)

print(f"\nVerification (first prediction for each task):")
for i, name in enumerate(output_cols):
    print(f"  {name}: Original={original_mean[0, i]:.2f}, Loaded={loaded_mean[0, i]:.2f}")

## 9. Comparison: MTGP vs Single-Task GP

MTGP learns correlations between outputs, which can improve predictions when outputs are related.

Key advantages:
- Shares information across tasks
- More data-efficient
- Captures output correlations
- Natural uncertainty quantification

In [ ]:
# Summary metrics
print("=== MTGP Performance Summary ===")
print("\n| Task | RMSE | MAE | R² |")
print("|------|------|-----|-------|")
for name in output_cols:
    m = metrics[name]
    print(f"| {name} | {m['rmse']:.2f} | {m['mae']:.2f} | {m['r2']:.3f} |")

## Summary

In this notebook, you learned how to:

1. **Load and explore** multi-output AD process data
2. **Configure** MTGP with Linear Model of Coregionalization
3. **Train** the model with variational inference
4. **Predict** with 95% confidence intervals
5. **Analyze** uncertainty and coverage
6. **Save and load** trained models

### Key Takeaways

- MTGP is ideal when you need **uncertainty quantification**
- It works well with **limited training data**
- Outputs should be **correlated** for best results
- 95% CI provides **reliability bounds** for predictions

### Next Steps

- Try different numbers of latent functions
- Experiment with different kernel functions
- Compare with LSTM for point predictions
- Use uncertainty for decision-making in control